## BenchmarkLoader

The cell below shows the code for loading a benchmark from a configuration file. Click [here](benchmark_configs/test_config.yaml) to see the config file.

Loading a benchmark performs all necessary processing steps including loading the data and generating benchmark tasks. To allow for downloading public 
data which requires attribution, downloading data from remote sources is supported. when a remote source is used, the proper credits are displayed to the
user.

In [1]:
from ppm_benchmark.core.benchmark_loader import BenchmarkLoader


loader = BenchmarkLoader()

benchmark = loader.load_from_config('ppm_benchmark/benchmark_configs/test_config.yaml')

## Loading Benchmark & Tasks

When a benchmark is generated from a config file it is automatically saved to disk. The `.load_from_folder()` method can be used to retrieve the configured benchmark.
Each benchmark revolves around several tasks on which performance can be measured. The `.get_tasks()` method returns all task names, these can be used to retrieve the training and testing data.

In [2]:
import random
from ppm_benchmark.core.benchmark_loader import BenchmarkLoader


loader = BenchmarkLoader()
benchmark = loader.load_from_folder('test_benchmark')

tasks = benchmark.get_tasks()
tasks 

## Getting Task Data

The cell below shows the code for retrieving the training and test data. This simple example is a classification task for the last activity. For this example the 'prediction' is a randomly selected activity.

In [3]:
task = benchmark.load_task('test_task')

preds = dict()
train_df = task.get_train_data()
test_df = task.get_test_data()
targets = train_df['concept:name'].unique().tolist()

for index, row in test_df.iterrows():
        pred = random.choice(targets)
        case_id = row['case:concept:name']
        preds[case_id] = pred
preds

## Evaluation

Evaluation is done for each individual task. The evaluation functions are specified in the config and will be displayed to the user when running the code below.

In [4]:
benchmark.evaluate(task, preds)

## Experiments

The experiment class is implemented to help users in tracking PPM experiments. A run can be initialized to track a task. If the optional parameter `model_type=tensorflow` is given to the `init_run()` method, the returned run_tracker is a Tensorflow callback which supports automatic tracking. For other model types, the generic run_tracker can be used inside the training loop for tracking metrics. The code below simulates a situation for tracking the loss.

In [3]:
from ppm_benchmark.core.experiment import Experiment


experiment = Experiment()
experiment.new_experiment('test_experiment')

task = benchmark.load_task('test_task')
run_tracker, run = experiment.init_run(task, 'test_run')

for i in range(0, 10):
        run_tracker.epoch_end({'loss': 0})

run_tracker.train_end()
run_tracker.evaluate(preds)


## Retrieving Run Data

using the `to_dict()` method on a run object returns all tracked run data in a dictionary format. This can easily be converted into a DataFrame for further analysis.

In [4]:
run.to_dict()